# Confiabilidade de viga mista

## Importando bibliotecas

In [26]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt
import pandas as pd

## Tabela de perfis

In [47]:
tabela = pd.read_excel('Catalogo-Tabela-Bitolas-Editado-2.xlsx', sheet_name=5)
display(tabela)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,espessura,espessura.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,eixo_x_x,eixo_x_x.1,...,eixo_y_y,eixo_y_y.1,eixo_y_y.2,eixo_y_y.3,eixo_y_y.4,eixo_y_y.5,esbeltez,esbeltez.1,Unnamed: 20,Unnamed: 21
0,massa_linear,d,bf,tw,tf\nmm,h,d',area,Ix,Wx,...,ly,Wy,ry,Zy,rt,lt,Mesa,Alma,Cw,u\n
1,kg/m,mm,mm,mm,mm,mm,mm,cm2,cm4,cm3,...,cm4,cm3,cm,cm3,cm,cm4,bf/2.tf,d'/tw,cm6,m2/m
2,13,148,100,4.3,4.9,138,118,16.6,635,85.8,...,82,16.4,2.22,25.5,2.6,1.72,10.2,27.49,4181,0.67
3,18,153,102,5.8,7.1,139,119,23.4,939,122.8,...,126,24.7,2.32,38.5,2.69,4.34,7.18,20.48,6683,0.69
4,22.5,152,152,5.8,6.6,139,119,29,1229,161.7,...,387,50.9,3.65,77.9,4.1,4.75,11.52,20.48,20417,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,154.2,623,229,14,24.9,573,541,196.5,125783,4038,...,4999,436.6,5.04,683.3,5.94,303.29,4.6,38.66,4456995,2.11
106,155,611,324,12.7,19,573,541,198.1,129583,4241.7,...,10783,665.6,7.38,1022.6,8.53,200.77,8.53,42.6,9436714,2.47
107,174,616,325,14,21.6,573,541,222.8,147754,4797.2,...,12374,761.5,7.45,1171.1,8.58,286.88,7.52,38.63,10915665,2.48
108,195,622,327,15.4,24.4,573,541,250.1,168484,5417.5,...,14240,870.9,7.55,"1341 ,0",8.66,405.29,6.7,35.14,12695302,2.49


## Definindo funções

In [27]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    amostra: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [28]:
espessura_laje: float = 10 # cm 
desvpad_espessura_laje: float = 0.06 * espessura_laje # cm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [29]:
limite_escoamento_perfil: float = 250 * 1.08 # MPa
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil # MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [30]:
limite_escoamento_armadura: float = 500 * 1.08 # MPa
desvpad_escoamento_armadura: float = 0.08 * limite_escoamento_armadura #MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [31]:
resistencia_concreto: float = 20 * 1.17 # MPa
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto # MPa

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [32]:
diametro_rebite: float = 1.59 # cm
desvpad_diametro_rebite: float = 0.01 * diametro_rebite # cm

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [33]:
resistencia_rebite: float = 415*1.06 # MPa
desvpad_resistencia_rebite: float = 0.06 * resistencia_rebite # MPa

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [34]:
carga_permanente_g1: float = 7.6 # kN/m
carga_permanente_g2: float = 5 # kN/m

variacao_carga: list = [3, 11]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9 # m
espacamento_laje: float = 2.8 # m

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * vao_laje ** 2) / 8 for i in range(len(carga_distribuida_projeto))]

altura_laje_ht: float = 45 # cm
coeficiente_ya1: float = 1.1
x: float = 10 # cm

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_laje_ht / 2) + espessuras_laje[j] - (x / 2)))) # cm²
    index_matriz += 1
    
print(areas_minimas) 


[[14.501706811421203, 14.714592360246758, 16.527154564041265, 13.237763033229193, 14.30352763694368, 15.7370359710694, 14.82517757800327, 15.223957091043225, 12.901392691399847, 13.264448958988302, 15.329839055900857, 14.018189125715487, 13.399596938477718, 13.4211867352516, 14.659567999091072, 14.050729576009575, 13.151908355759925, 15.471249566326218, 13.550773603898321, 12.970022230901499, 13.647940206102508, 14.263074523973911, 14.434153494489383, 13.94771451555874, 14.982503223382325, 16.38315714908526, 13.789342939557518, 15.779353221057972, 16.87637480527148, 15.240960835942099, 14.138692845892766, 14.949339534796422, 13.73187962562532, 15.061422807616497, 14.02931477412258, 14.010189768493778, 14.23437230983934, 15.911434467876568, 16.645589284442362, 16.044220353751946, 15.578604661104372, 13.576779591866007, 12.37950805781748, 14.265648085317807, 15.321825199443706, 14.906882882926437, 14.081525693315136, 14.561347797884837, 18.135135448557637, 13.882849011270569, 14.11718012

## Flambagem da seção

In [35]:
# Classificação da seção quanto a flambagem

altura_perfil: float = 404 # mm
espessura_perfil: float = 7.6 # mm
modulo_elasticidade_perfil: float = 200000 # MPa

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

countador_compacto: int = 0
countador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        countador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        countador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {countador_compacto}')
print(f'Semi-compactas: {countador_semicompacto}')
        

Compactas: 1000
Semi-compactas: 0
